In [41]:
import fitz
import os
import numpy as np
import json
from openai import OpenAI
import torch
from sentence_transformers import SentenceTransformer

In [42]:
from dotenv import load_dotenv
load_dotenv()

True

In [43]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

In [44]:
# Load the pre-trained model
embedder = SentenceTransformer("all-MiniLM-L6-v2") # Fazer o embedding usando esse

In [45]:
pdf_path = "/home/patrick/rag_from_scratch/AI_Information.pdf"

In [46]:
def extract_text_from_pdf(pdf_path):
    mypdf = fitz.open(pdf_path)
    all_text = "" # Initizalizing an empty string to store the extarcted text

    # Iterating through each page in the pdf
    for page_num in range(mypdf.page_count):
        page = mypdf[page_num]
        text = page.get_text("text")
        all_text += text

    return all_text

In [47]:
extracted_text = extract_text_from_pdf(pdf_path)

In [48]:
def chunk_text(text, n, overlap):
    chunks = []
    for i in range(0, len(text), n - overlap):
        print(text[i:i+n])
        print("-----")
        chunks.append(text[i:i+n])
    return chunks

# Define different chunk sizes to evaluate
chunk_sizes = [128, 256, 512]

# Create a dictionary to store text chunks for each chunk size
text_chunks_dict = {size: chunk_text(extracted_text, size, size // 5) for size in chunk_sizes}

# Print the number of chunks created for each chunk size
for size, chunks in text_chunks_dict.items():
    print(f"Chunk Size: {size}, Number of Chunks: {len(chunks)}")

Understanding Artificial Intelligence 
Chapter 1: Introduction to Artificial Intelligence 
Artificial intelligence (AI) refers t
-----
ntelligence (AI) refers to the ability of a digital computer or computer-controlled robot 
to perform tasks commonly associated 
-----
asks commonly associated with intelligent beings. The term is frequently applied to 
the project of developing systems endowed w
-----
eloping systems endowed with the intellectual processes characteristic of 
humans, such as the ability to reason, discover meani
-----
to reason, discover meaning, generalize, or learn from past 
experience. Over the past few decades, advancements in computing po
-----
ancements in computing power and data availability 
have significantly accelerated the development and deployment of AI. 
Histor
-----
deployment of AI. 
Historical Context 
The idea of artificial intelligence has existed for centuries, often depicted in myths an
-----
ften depicted in myths and fiction. 
However, the forma

In [49]:
from tqdm import tqdm

In [50]:
def create_embeddings(text):
    response = embedder.encode(text)
    return response

chunk_embeddings_dict = {size: create_embeddings(chunks) for size, chunks in tqdm(text_chunks_dict.items(), desc="Generating Embeddings")}


Generating Embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Embeddings: 100%|██████████| 3/3 [00:09<00:00,  3.10s/it]


In [51]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [52]:
def semantic_search(query, text_chunks, embeddings, k=5):
    query_embedding = create_embeddings(query)
    similarity_scores = []

    for i, chunk_embedding in enumerate(embeddings):
        similarity_score = cosine_similarity(np.array(query_embedding), np.array(chunk_embedding))
        similarity_scores.append((i, similarity_score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_indices = [index for index, _ in similarity_scores[:k]]
    return [text_chunks[index] for index in top_indices]

In [53]:
# Load the validation data from a JSON file
with open('val.json') as f:
    data = json.load(f)

# Extract the first query from the validation data
query = data[0]['question']

# Retrieve relevant chunks for each chunk size
retrieved_chunks_dict = {size: semantic_search(query, text_chunks_dict[size], chunk_embeddings_dict[size]) for size in chunk_sizes}

# Print retrieved chunks for chunk size 256
print(retrieved_chunks_dict[256])

['opment include: \nExplainable AI (XAI) \nExplainable AI (XAI) aims to make AI systems more transparent and understandable. XAI \ntechniques are being developed to provide insights into how AI models make decisions, \nenhancing trust and accountability. \nAI at ', 'ing techniques. \nExplainable AI (XAI) \nExplainable AI (XAI) aims to make AI systems more transparent and understandable. Research in \nXAI focuses on developing methods for explaining AI decisions, enhancing trust, and improving \naccountability. \nAI and Neu', 'explainability is \ncrucial for building trust and accountability. \n \n \nPrivacy and Security \nAI systems often rely on large amounts of data, raising concerns about privacy and data security. \nProtecting sensitive information and ensuring responsible data h', 'ability \nTransparency and explainability are essential for building trust in AI systems. Explainable AI (XAI) \ntechniques aim to make AI decisions more understandable, enabling users to assess their \

In [54]:
print(retrieved_chunks_dict[128])

['option \nacross various domains. Key trends and areas of development include: \nExplainable AI (XAI) \nExplainable AI (XAI) aims to', 's. \nExplainable AI (XAI) \nExplainable AI (XAI) aims to make AI systems more transparent and understandable. Research in \nXAI foc', 'are essential for building trust in AI systems. Explainable AI (XAI) \ntechniques aim to make AI decisions more understandable, e', ' and interpretable deep learning models, as well \nas exploring new architectures and training techniques. \nExplainable AI (XAI) ', ' systems. \nChapter 20: Building Trust in AI \nTransparency and Explainability \nTransparency and explainability are key to buildin']


In [55]:
system_prompt = "You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly from the provided context, respond with: 'I do not have enough information to answer that.'"

def generate_response(system_prompt, query, retrieved_chunks, model="gpt-3.5-turbo-1106"):
    context = "\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(retrieved_chunks)])
    
    # Create the user prompt by combining the context and the query
    user_prompt = f"{context}\n\nQuestion: {query}"
    
    response = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    return response

In [56]:
# Generate AI responses for each chunk size
ai_responses_dict = {size: generate_response(system_prompt, query, retrieved_chunks_dict[size]) for size in chunk_sizes}

# Print the response for chunk size 256
print(ai_responses_dict[256].choices[0].message.content)

'Explainable AI' (XAI) aims to make AI systems more transparent and understandable. It is considered important because it enhances trust and accountability by providing insights into how AI models make decisions, enabling users to assess their fairness and accuracy.


In [57]:
import json
import asyncio
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, SimilarityFunction

# Initialize the SentenceTransformer model and set the similarity function
model = SentenceTransformer("all-MiniLM-L6-v2")
model.similarity_fn_name = SimilarityFunction.DOT

In [58]:
async def process_validation_data(chunk_size):
    system_prompt = """You are an AI assistant that strictly answers based on the given context. If the answer cannot be derived directly and exactly from the provided context, respond with: 'I do not have enough information to answer that.'
    First think about the keywords from the question and then use them to elaborate the answer.
    The response needs to be just the answer sentence
    
    """
    # Load the validation data from the JSON file
    with open('val.json') as f:
        data = json.load(f)

    # List to store the results for each sample
    results = []

    # Iterate over each example in the validation data
    for idx, item in enumerate(data):
        query = item['question']
        ideal_answer = item['ideal_answer']
        
        # Retrieve the top k most relevant context chunks
        retrieved_chunks_dict = {size: semantic_search(query, text_chunks_dict[size], chunk_embeddings_dict[size]) for size in chunk_sizes}
        
        # Generate the AI response using the system prompt and the user prompt
        ai_responses_dict = {size: generate_response(system_prompt, query, retrieved_chunks_dict[size]) for size in chunk_sizes}
        
        # Evaluate similarity using SentenceTransformer
        # Encode the AI response and ideal answer
        ai_response = ai_responses_dict[chunk_size].choices[0].message.content
        embedding_response = model.encode([ai_response])
        embedding_ideal = model.encode([ideal_answer])
        # Compute similarity score (result is a 1x1 matrix; extract the single value)
        similarity_matrix = model.similarity(embedding_response, embedding_ideal)
        score = similarity_matrix[0][0].numpy()
        
        # Prepare the result dictionary with dynamic context columns
        result = {
            "Query": query,
            "Ideal Answer": ideal_answer,
            "AI Response": ai_response,
            "Score": score
        }        
        # Append the result to the list
        results.append(result)

    # Create a DataFrame from the results
    df = pd.DataFrame(results)
    return df

In [59]:
import nest_asyncio
nest_asyncio.apply()

In [60]:
# Run the asynchronous function and store the DataFrame in df
df = asyncio.run(process_validation_data(chunk_size=512))
df

,Query,Ideal Answer,AI Response,Score
0,What is 'Explainable AI' and why is it conside...,Explainable AI (XAI) aims to make AI systems m...,Explainable AI (XAI) aims to make AI systems m...,0.9616332
1,Can AI be used to predict earthquakes?,I don't have enough information to answer that.,I do not have enough information to answer that.,0.9543278
2,What are some of the ethical concerns related ...,I don't have enough information to answer that.,AI systems can inherit and amplify biases pres...,0.14903635
3,How does AI contribute to personalized medicine?,AI enables personalized medicine by analyzing ...,AI contributes to personalized medicine by ana...,0.9515934
4,Does the document mention any specific compani...,I don't have enough information to answer that.,I do not have enough information to answer that.,0.9543278
5,What is the role of AI in smart grids?,AI optimizes energy distribution in smart grid...,AI optimizes energy management in smart cities...,0.7971191
6,"Can AI write a complete, original novel?",I don't have enough information to answer that.,"AI can write articles, generate content, and c...",0.068094164
7,What is a 'cobot'?,It mentions collaborative settings (cobots) in...,A 'cobot' is a collaborative robot that works ...,0.68255216
8,What is Direct Air Capture (DAC) used for?,DAC technology removes CO2 directly from the a...,I do not have enough information to answer that.,0.04811383
9,Is AI currently being used to control nuclear ...,I don't have enough information to answer that.,I do not have enough information to answer that.,0.9543278


In [40]:
for k in [128, 256, 512]:
    print(asyncio.run(process_validation_data(chunk_size=k))['Score'].mean())

0.5760783195495606
0.5732148170471192
0.655241584777832
